In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,SimpleRNN
import tensorflow as tf


In [2]:
df = pd.read_csv(r"C:\Users\rosem\Downloads\Paper_works\CSVs\Feature_engg_Paper2.csv")
df = df[df['Region'].isin(['CAL','TEX'])]
df

,Date,Hour,Demand,Net_Generation,Total_Interchange,Forecasted_Demand,Coal_Gen,Gas_Gen,Nuclear_Gen,Hydro_Gen,...,Rolling_Mean_24H,Rolling_Mean_3H_Emission,Rolling_Mean_24H_Emission,Total_Gen,Renewable_Pct,Fossil_Pct,Renewable_Demand,Fossil_Demand,Demand_diff1,Demand_diff24
0,2018-07-01 17:00:00,12.5,28976.042,21138.304,-6528.677,28386.220,8.764,7725.951,2261.073,3089.741,...,42604.509,27096.582,17297.042,17521.203,42.950,44.145,1244530.613,1279144.078,89.458,172.875
1,2018-07-01 18:00:00,12.5,29065.500,21217.885,-6554.521,28486.010,8.792,7761.917,2261.062,3110.594,...,41847.195,8244.812,17052.778,17577.073,42.927,44.209,1247693.394,1284966.639,89.458,172.875
2,2018-07-01 19:00:00,12.5,29154.958,21297.467,-6580.365,28585.800,8.819,7797.882,2261.052,3131.446,...,45629.697,24266.186,19327.269,17632.943,42.904,44.273,1250856.313,1290788.823,89.458,172.875
3,2018-07-01 20:00:00,12.5,29244.417,21377.049,-6606.208,28685.590,8.847,7833.847,2261.042,3152.299,...,48609.446,6436.826,20904.933,17688.812,42.881,44.337,1254019.366,1296610.635,89.458,172.875
4,2018-07-01 21:00:00,12.5,29333.875,21456.630,-6632.052,28785.380,8.875,7869.812,2261.031,3173.151,...,49142.567,6326.084,21205.564,17744.682,42.858,44.400,1257182.553,1302432.078,89.458,172.875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525055,2023-06-29 20:00:00,12.5,64967.778,65025.979,58.042,65732.083,8989.486,27552.910,4647.181,32.896,...,49409.849,30361.664,19267.173,46077.069,10.607,79.307,689127.079,5152407.218,-65.642,-1360.549
525056,2023-06-29 21:00:00,12.5,64902.135,64963.474,61.177,65699.302,8993.729,27593.401,4647.292,33.224,...,46137.266,13730.128,18243.586,46106.208,10.566,79.354,685784.748,5150245.412,-65.642,-1414.266
525057,2023-06-29 22:00:00,12.5,64836.493,64900.969,64.312,65666.521,8997.972,27633.892,4647.403,33.552,...,45454.091,12961.713,16905.630,46135.347,10.526,79.401,682451.108,5148073.607,-65.642,-1467.983
525058,2023-06-29 23:00:00,12.5,64770.851,64838.464,67.448,65633.740,9002.215,27674.384,4647.514,33.880,...,39779.804,15939.369,14414.641,46164.486,10.485,79.448,679126.143,5145891.821,-65.642,-1521.700


# 80-20

In [4]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
# from sklearn.preprocessing import MinMaxScaler
# import numpy as np
# import pandas as pd
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

region_code = 'CAL'   # or 'TEX' for Texas
epochs_list = [1, 2, 5, 10]
results = []

# --- Use your full data, no grouping/aggregation ---
region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df = region_df.sort_values('Date').reset_index(drop=True)

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_scaled, y_scaled)
split = int(0.8 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

for epoch in epochs_list:
    model = Sequential([
        LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(2)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)
    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    mae_demand = mean_absolute_error(y_true[:, 0], y_pred[:, 0])
    rmse_demand = np.sqrt(mean_squared_error(y_true[:, 0], y_pred[:, 0]))
    mape_demand = mean_absolute_percentage_error(y_true[:, 0], y_pred[:, 0])
    mae_co2 = mean_absolute_error(y_true[:, 1], y_pred[:, 1])
    rmse_co2 = np.sqrt(mean_squared_error(y_true[:, 1], y_pred[:, 1]))
    mape_co2 = mean_absolute_percentage_error(y_true[:, 1], y_pred[:, 1])

    results.append({
        'Region': region_code,
        'Epoch': epoch,
        'MAE_Demand': mae_demand,
        'RMSE_Demand': rmse_demand,
        'MAPE_Demand(%)': mape_demand,
        'MAE_CO2': mae_co2,
        'RMSE_CO2': rmse_co2,
        'MAPE_CO2(%)': mape_co2
    })

results_df = pd.DataFrame(results)
print(results_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


274/274 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


274/274 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


274/274 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


274/274 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
  Region  Epoch  MAE_Demand  RMSE_Demand  MAPE_Demand(%)     MAE_CO2  \
0    CAL      1  765.031135   940.916000        2.573330  204.353924   
1    CAL      2  389.229736   472.010246        1.299197  224.041498   
2    CAL      5  176.320161   255.536672        0.565540  176.145735   
3    CAL     10  250.870155   285.450241        0.817473  185.174937   

     RMSE_CO2  MAPE_CO2(%)  
0  252.736606     4.180566  
1  261.016209     4.722971  
2  215.516196     3.639422  
3  230.066275     3.429333  


# 75-25

In [5]:
from sklearn.metrics import mean_absolute_error, mean_squared_error


def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

region_code = 'CAL'   # or 'TEX' for Texas
epochs_list = [1, 2, 5, 10]
results = []

# --- Use your full data, no grouping/aggregation ---
region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df = region_df.sort_values('Date').reset_index(drop=True)

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_scaled, y_scaled)
split = int(0.75 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

for epoch in epochs_list:
    model = Sequential([
        LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(2)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)
    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    mae_demand = mean_absolute_error(y_true[:, 0], y_pred[:, 0])
    rmse_demand = np.sqrt(mean_squared_error(y_true[:, 0], y_pred[:, 0]))
    mape_demand = mean_absolute_percentage_error(y_true[:, 0], y_pred[:, 0])
    mae_co2 = mean_absolute_error(y_true[:, 1], y_pred[:, 1])
    rmse_co2 = np.sqrt(mean_squared_error(y_true[:, 1], y_pred[:, 1]))
    mape_co2 = mean_absolute_percentage_error(y_true[:, 1], y_pred[:, 1])

    results.append({
        'Region': region_code,
        'Epoch': epoch,
        'MAE_Demand': mae_demand,
        'RMSE_Demand': rmse_demand,
        'MAPE_Demand(%)': mape_demand,
        'MAE_CO2': mae_co2,
        'RMSE_CO2': rmse_co2,
        'MAPE_CO2(%)': mape_co2
    })

results_df = pd.DataFrame(results)
print(results_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


342/342 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


342/342 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


342/342 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


342/342 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  Region  Epoch  MAE_Demand  RMSE_Demand  MAPE_Demand(%)     MAE_CO2  \
0    CAL      1  452.745590   565.376855        1.525404  233.382924   
1    CAL      2  615.772314   732.058239        2.113910  177.478503   
2    CAL      5  157.994897   225.247661        0.508859  179.516823   
3    CAL     10  213.723218   267.006626        0.686355  254.627428   

     RMSE_CO2  MAPE_CO2(%)  
0  281.565911     5.077240  
1  219.857412     3.908038  
2  216.992924     4.000956  
3  283.957982     5.578378  


In [7]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

region_code = 'CAL'   # or 'TEX' for Texas
epochs_list = [1, 2, 5, 10]
results = []

# --- Use your full data, no grouping/aggregation ---
region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df = region_df.sort_values('Date').reset_index(drop=True)

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_scaled, y_scaled)
split = int(0.7 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

for epoch in epochs_list:
    model = Sequential([
        LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(2)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)
    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    mae_demand = mean_absolute_error(y_true[:, 0], y_pred[:, 0])
    rmse_demand = np.sqrt(mean_squared_error(y_true[:, 0], y_pred[:, 0]))
    mape_demand = mean_absolute_percentage_error(y_true[:, 0], y_pred[:, 0])
    mae_co2 = mean_absolute_error(y_true[:, 1], y_pred[:, 1])
    rmse_co2 = np.sqrt(mean_squared_error(y_true[:, 1], y_pred[:, 1]))
    mape_co2 = mean_absolute_percentage_error(y_true[:, 1], y_pred[:, 1])

    results.append({
        'Region': region_code,
        'Epoch': epoch,
        'MAE_Demand': mae_demand,
        'RMSE_Demand': rmse_demand,
        'MAPE_Demand(%)': mape_demand,
        'MAE_CO2': mae_co2,
        'RMSE_CO2': rmse_co2,
        'MAPE_CO2(%)': mape_co2
    })

results_df = pd.DataFrame(results)
print(results_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


411/411 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
  Region  Epoch  MAE_Demand  RMSE_Demand  MAPE_Demand(%)     MAE_CO2  \
0    CAL      1  655.464675   837.255182        2.253452  224.220631   
1    CAL      2  437.372985   582.741505        1.483881  247.984798   
2    CAL      5  306.594937   375.514398        0.999819  267.594726   
3    CAL     10  258.833208   304.138362        0.866819  281.991604   

     RMSE_CO2  MAPE_CO2(%)  
0  288.166423     5.000464  
1  293.336551     5.753386  
2  303.455486     5.894369  
3  324.086906     6.284939  


In [9]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

region_code = 'CAL'   # or 'TEX' for Texas
epochs_list = [1, 2, 5, 10]
results = []

# --- Use your full data, no grouping/aggregation ---
region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df = region_df.sort_values('Date').reset_index(drop=True)

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_scaled, y_scaled)
split = int(0.65 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

for epoch in epochs_list:
    model = Sequential([
        LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(2)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)
    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    mae_demand = mean_absolute_error(y_true[:, 0], y_pred[:, 0])
    rmse_demand = np.sqrt(mean_squared_error(y_true[:, 0], y_pred[:, 0]))
    mape_demand = mean_absolute_percentage_error(y_true[:, 0], y_pred[:, 0])
    mae_co2 = mean_absolute_error(y_true[:, 1], y_pred[:, 1])
    rmse_co2 = np.sqrt(mean_squared_error(y_true[:, 1], y_pred[:, 1]))
    mape_co2 = mean_absolute_percentage_error(y_true[:, 1], y_pred[:, 1])

    results.append({
        'Region': region_code,
        'Epoch': epoch,
        'MAE_Demand': mae_demand,
        'RMSE_Demand': rmse_demand,
        'MAPE_Demand(%)': mape_demand,
        'MAE_CO2': mae_co2,
        'RMSE_CO2': rmse_co2,
        'MAPE_CO2(%)': mape_co2
    })

results_df = pd.DataFrame(results)
print(results_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


479/479 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


479/479 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


479/479 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


479/479 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step
  Region  Epoch   MAE_Demand  RMSE_Demand  MAPE_Demand(%)     MAE_CO2  \
0    CAL      1  1021.532143  1250.423019        3.499196  289.563371   
1    CAL      2   436.678546   555.039156        1.478511  230.862944   
2    CAL      5   277.826853   368.430893        0.933842  432.808036   
3    CAL     10   128.275644   182.634307        0.425096  400.335355   

     RMSE_CO2  MAPE_CO2(%)  
0  350.028565     6.335821  
1  276.677424     5.067166  
2  479.806583     9.164571  
3  440.543839     8.416329  


In [11]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

region_code = 'CAL'   # or 'TEX' for Texas
epochs_list = [1, 2, 5, 10]
results = []

# --- Use your full data, no grouping/aggregation ---
region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df = region_df.sort_values('Date').reset_index(drop=True)

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_scaled, y_scaled)
split = int(0.6 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

for epoch in epochs_list:
    model = Sequential([
        LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(2)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)
    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    mae_demand = mean_absolute_error(y_true[:, 0], y_pred[:, 0])
    rmse_demand = np.sqrt(mean_squared_error(y_true[:, 0], y_pred[:, 0]))
    mape_demand = mean_absolute_percentage_error(y_true[:, 0], y_pred[:, 0])
    mae_co2 = mean_absolute_error(y_true[:, 1], y_pred[:, 1])
    rmse_co2 = np.sqrt(mean_squared_error(y_true[:, 1], y_pred[:, 1]))
    mape_co2 = mean_absolute_percentage_error(y_true[:, 1], y_pred[:, 1])

    results.append({
        'Region': region_code,
        'Epoch': epoch,
        'MAE_Demand': mae_demand,
        'RMSE_Demand': rmse_demand,
        'MAPE_Demand(%)': mape_demand,
        'MAE_CO2': mae_co2,
        'RMSE_CO2': rmse_co2,
        'MAPE_CO2(%)': mape_co2
    })

results_df = pd.DataFrame(results)
print(results_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
  Region  Epoch  MAE_Demand  RMSE_Demand  MAPE_Demand(%)     MAE_CO2  \
0    CAL      1  657.102931   810.746520        2.191914  368.174328   
1    CAL      2  534.270795   650.258551        1.755141  314.110072   
2    CAL      5  195.977857   239.582740        0.637382  328.304875   
3    CAL     10  127.659290   160.901990        0.423689  378.207405   

     RMSE_CO2  MAPE_CO2(%)  
0  421.824945     8.242413  
1  367.676981     6.801819  
2  372.565185     7.188161  
3  430.748427     7.987652  


In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

region_code = 'TEX'   # or 'TEX' for Texas
epochs_list = [1, 2, 5, 10]
results = []

# --- Use your full data, no grouping/aggregation ---
region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df = region_df.sort_values('Date').reset_index(drop=True)

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_scaled, y_scaled)
split = int(0.8 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

for epoch in epochs_list:
    model = Sequential([
        LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(2)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)
    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    mae_demand = mean_absolute_error(y_true[:, 0], y_pred[:, 0])
    rmse_demand = np.sqrt(mean_squared_error(y_true[:, 0], y_pred[:, 0]))
    mape_demand = mean_absolute_percentage_error(y_true[:, 0], y_pred[:, 0])
    mae_co2 = mean_absolute_error(y_true[:, 1], y_pred[:, 1])
    rmse_co2 = np.sqrt(mean_squared_error(y_true[:, 1], y_pred[:, 1]))
    mape_co2 = mean_absolute_percentage_error(y_true[:, 1], y_pred[:, 1])

    results.append({
        'Region': region_code,
        'Epoch': epoch,
        'MAE_Demand': mae_demand,
        'RMSE_Demand': rmse_demand,
        'MAPE_Demand(%)': mape_demand,
        'MAE_CO2': mae_co2,
        'RMSE_CO2': rmse_co2,
        'MAPE_CO2(%)': mape_co2
    })

results_df = pd.DataFrame(results)
print(results_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


274/274 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  Region  Epoch  MAE_Demand  RMSE_Demand  MAPE_Demand(%)     MAE_CO2  \
0    TEX      1  500.872947   645.254161        1.058198  691.684488   
1    TEX      2  419.898412   544.890733        0.826503  803.202026   
2    TEX      5  284.314935   371.637810        0.597990  683.751943   
3    TEX     10  257.158568   367.949984        0.498200  824.095289   

      RMSE_CO2  MAPE_CO2(%)  
0   915.544972     5.156128  
1  1031.517544     5.980817  
2   873.508170     4.972774  
3  1043.312800     5.963689  


In [15]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

region_code = 'TEX'   # or 'TEX' for Texas
epochs_list = [1, 2, 5, 10]
results = []

# --- Use your full data, no grouping/aggregation ---
region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df = region_df.sort_values('Date').reset_index(drop=True)

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_scaled, y_scaled)
split = int(0.75 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

for epoch in epochs_list:
    model = Sequential([
        LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(2)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)
    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    mae_demand = mean_absolute_error(y_true[:, 0], y_pred[:, 0])
    rmse_demand = np.sqrt(mean_squared_error(y_true[:, 0], y_pred[:, 0]))
    mape_demand = mean_absolute_percentage_error(y_true[:, 0], y_pred[:, 0])
    mae_co2 = mean_absolute_error(y_true[:, 1], y_pred[:, 1])
    rmse_co2 = np.sqrt(mean_squared_error(y_true[:, 1], y_pred[:, 1]))
    mape_co2 = mean_absolute_percentage_error(y_true[:, 1], y_pred[:, 1])

    results.append({
        'Region': region_code,
        'Epoch': epoch,
        'MAE_Demand': mae_demand,
        'RMSE_Demand': rmse_demand,
        'MAPE_Demand(%)': mape_demand,
        'MAE_CO2': mae_co2,
        'RMSE_CO2': rmse_co2,
        'MAPE_CO2(%)': mape_co2
    })

results_df = pd.DataFrame(results)
print(results_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


342/342 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


342/342 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


342/342 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


342/342 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
  Region  Epoch  MAE_Demand  RMSE_Demand  MAPE_Demand(%)      MAE_CO2  \
0    TEX      1  608.598779   798.961724        1.194526   707.228094   
1    TEX      2  610.292624   805.057167        1.181940  1242.247397   
2    TEX      5  324.129318   454.411546        0.622691  1071.176287   
3    TEX     10  270.540282   360.419111        0.536528  1072.511026   

      RMSE_CO2  MAPE_CO2(%)  
0   904.144696     5.025213  
1  1434.109456     8.303633  
2  1273.991433     6.909573  
3  1290.228230     7.257859  


In [16]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

region_code = 'TEX'   # or 'TEX' for Texas
epochs_list = [1, 2, 5, 10]
results = []

# --- Use your full data, no grouping/aggregation ---
region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df = region_df.sort_values('Date').reset_index(drop=True)

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_scaled, y_scaled)
split = int(0.7 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

for epoch in epochs_list:
    model = Sequential([
        LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(2)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)
    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    mae_demand = mean_absolute_error(y_true[:, 0], y_pred[:, 0])
    rmse_demand = np.sqrt(mean_squared_error(y_true[:, 0], y_pred[:, 0]))
    mape_demand = mean_absolute_percentage_error(y_true[:, 0], y_pred[:, 0])
    mae_co2 = mean_absolute_error(y_true[:, 1], y_pred[:, 1])
    rmse_co2 = np.sqrt(mean_squared_error(y_true[:, 1], y_pred[:, 1]))
    mape_co2 = mean_absolute_percentage_error(y_true[:, 1], y_pred[:, 1])

    results.append({
        'Region': region_code,
        'Epoch': epoch,
        'MAE_Demand': mae_demand,
        'RMSE_Demand': rmse_demand,
        'MAPE_Demand(%)': mape_demand,
        'MAE_CO2': mae_co2,
        'RMSE_CO2': rmse_co2,
        'MAPE_CO2(%)': mape_co2
    })

results_df = pd.DataFrame(results)
print(results_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
  Region  Epoch  MAE_Demand  RMSE_Demand  MAPE_Demand(%)      MAE_CO2  \
0    TEX      1  652.035713   818.702915        1.387842   708.070046   
1    TEX      2  669.241777   865.130971        1.339668   691.304612   
2    TEX      5  473.212389   622.326228        0.937345  1082.042322   
3    TEX     10  325.009783   426.728820        0.660084  1145.664770   

      RMSE_CO2  MAPE_CO2(%)  
0   898.866202     4.832974  
1   880.299138     4.863367  
2  1317.714407     6.986695  
3  1391.533970     7.210452  


In [17]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

region_code = 'TEX'   # or 'TEX' for Texas
epochs_list = [1, 2, 5, 10]
results = []

# --- Use your full data, no grouping/aggregation ---
region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df = region_df.sort_values('Date').reset_index(drop=True)

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_scaled, y_scaled)
split = int(0.65 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

for epoch in epochs_list:
    model = Sequential([
        LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(2)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)
    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    mae_demand = mean_absolute_error(y_true[:, 0], y_pred[:, 0])
    rmse_demand = np.sqrt(mean_squared_error(y_true[:, 0], y_pred[:, 0]))
    mape_demand = mean_absolute_percentage_error(y_true[:, 0], y_pred[:, 0])
    mae_co2 = mean_absolute_error(y_true[:, 1], y_pred[:, 1])
    rmse_co2 = np.sqrt(mean_squared_error(y_true[:, 1], y_pred[:, 1]))
    mape_co2 = mean_absolute_percentage_error(y_true[:, 1], y_pred[:, 1])

    results.append({
        'Region': region_code,
        'Epoch': epoch,
        'MAE_Demand': mae_demand,
        'RMSE_Demand': rmse_demand,
        'MAPE_Demand(%)': mape_demand,
        'MAE_CO2': mae_co2,
        'RMSE_CO2': rmse_co2,
        'MAPE_CO2(%)': mape_co2
    })

results_df = pd.DataFrame(results)
print(results_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


479/479 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


479/479 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


479/479 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


479/479 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
  Region  Epoch  MAE_Demand  RMSE_Demand  MAPE_Demand(%)      MAE_CO2  \
0    TEX      1  701.707452   911.394561        1.453702   653.068102   
1    TEX      2  848.534790  1042.582664        1.795109   725.296952   
2    TEX      5  386.542460   480.516706        0.822309   914.376346   
3    TEX     10  454.974737   606.655605        0.938540  1076.152991   

      RMSE_CO2  MAPE_CO2(%)  
0   830.319757     4.471865  
1   925.038723     5.119523  
2  1111.541973     5.987209  
3  1327.048338     7.151549  


In [18]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

region_code = 'TEX'   # or 'TEX' for Texas
epochs_list = [1, 2, 5, 10]
results = []

# --- Use your full data, no grouping/aggregation ---
region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df = region_df.sort_values('Date').reset_index(drop=True)

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_scaled, y_scaled)
split = int(0.6 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

for epoch in epochs_list:
    model = Sequential([
        LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(2)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)
    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    mae_demand = mean_absolute_error(y_true[:, 0], y_pred[:, 0])
    rmse_demand = np.sqrt(mean_squared_error(y_true[:, 0], y_pred[:, 0]))
    mape_demand = mean_absolute_percentage_error(y_true[:, 0], y_pred[:, 0])
    mae_co2 = mean_absolute_error(y_true[:, 1], y_pred[:, 1])
    rmse_co2 = np.sqrt(mean_squared_error(y_true[:, 1], y_pred[:, 1]))
    mape_co2 = mean_absolute_percentage_error(y_true[:, 1], y_pred[:, 1])

    results.append({
        'Region': region_code,
        'Epoch': epoch,
        'MAE_Demand': mae_demand,
        'RMSE_Demand': rmse_demand,
        'MAPE_Demand(%)': mape_demand,
        'MAE_CO2': mae_co2,
        'RMSE_CO2': rmse_co2,
        'MAPE_CO2(%)': mape_co2
    })

results_df = pd.DataFrame(results)
print(results_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
  Region  Epoch  MAE_Demand  RMSE_Demand  MAPE_Demand(%)      MAE_CO2  \
0    TEX      1  762.321228   961.095832        1.582127   743.743156   
1    TEX      2  806.732658  1049.796539        1.632132   778.366869   
2    TEX      5  545.687449   790.338992        1.057589  1369.046416   
3    TEX     10  551.737979   825.554287        1.047678  1129.083720   

      RMSE_CO2  MAPE_CO2(%)  
0   948.039039     5.113342  
1   964.011216     5.254727  
2  1662.584363     8.744528  
3  1397.200301     7.122101  
